## importing libraries

In [1]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2017-01-30 07:53:36'

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
import datetime
from pyspark.sql import Row, functions as F
from pyspark.sql import SQLContext
import pandas as pd

## path and repartition variable

In [4]:
#To read all the files in the working directory
##for meta campaign data and travelers data

import os
files_path_dir = "/Users/curiouspratik/Work/data"
list_files = os.listdir(files_path_dir)

rep=1
##content file path
content_files_path = "/Users/curiouspratik/Work/content_data/"

rep = 1

# importing meta campaign raw files

In [5]:
#Filtering based on the required campaign files 
tab_file_path = []
required_campaign_files = ['campaign_definition.tab', 'template_definition.tab','segment_module.tab',
                          'module_variable_definition.tab','segment_definition.tab','module_definition.tab','traveler.tab']

#we can remove the required_campaign_files , then it would read all the files in the directory
for files in list_files:
    if (files.endswith('.tab') and files in required_campaign_files):
         tab_file_path.append(files)       
print(tab_file_path)

['campaign_definition.tab', 'module_definition.tab', 'module_variable_definition.tab', 'segment_definition.tab', 'segment_module.tab', 'template_definition.tab', 'traveler.tab']


In [7]:
spark-submit --packages com.databricks:spark-csv_2.10:1.2.0 your_script.py

SyntaxError: invalid syntax (<ipython-input-7-26103259316d>, line 1)

In [6]:
#this code would create all the required dataframes dynamically 
for file_name in tab_file_path:
    data_framename = 'df'+(''.join([i.title() for i in file_name.split('.')[0].split('_')]))
    globals()[data_framename] = (sqlContext.read.format("com.databricks.spark.csv").option("delimiter","\t")
                                .option("header", "true").option("inferSchema", "true").load(files_path_dir+'/'+file_name))

Py4JJavaError: An error occurred while calling o23.load.
: java.lang.ClassNotFoundException: Failed to find data source: com.databricks.spark.csv. Please find packages at http://spark-packages.org
	at org.apache.spark.sql.execution.datasources.ResolvedDataSource$.lookupDataSource(ResolvedDataSource.scala:77)
	at org.apache.spark.sql.execution.datasources.ResolvedDataSource$.apply(ResolvedDataSource.scala:102)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:119)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:109)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
	at py4j.Gateway

In [6]:
#getting all the content files required

def file_name(row):
    join_key = row
    if( len(join_key.split('|')) > 1):
        return join_key.split('|')[1].split(';')[0].split('.')[0]
    
file_name_udf = udf(file_name,StringType())

file_names = (dfModuleVariableDefinition.select("var_source").distinct()
                .withColumn("file_name",file_name_udf('var_source')))

ls_files = file_names.select("file_name").filter(file_names.file_name.isNotNull()).distinct().flatMap(lambda x:x).collect()

In [7]:
#Reading Content Files Dynamically

frames_dict = {}

for file_name in ls_files:
    path = content_files_path + file_name + ".tab"
    key = file_name
    frames_dict[key] = (sc.broadcast(sqlContext.read.format("com.databricks.spark.csv").option("delimiter","\t")
                                .option("header", "true").option("inferSchema", "true").load(path).toPandas()))

In [8]:
#correcting the value of locale

def replace(x):
    return x.replace('_','-')


udf_replace = udf(replace,StringType())
dfTraveler = dfTraveler.withColumn("locale",udf_replace(dfTraveler.locale))
dfModuleDefinition = dfModuleDefinition.withColumn("locale",udf_replace(dfModuleDefinition.locale))
dfCampaignDefinition = dfCampaignDefinition.withColumn("locale",udf_replace(dfCampaignDefinition.locale))

In [9]:
## changing column name of the loyalty column as present in segment criteria

dfTraveler = dfTraveler.withColumnRenamed("loyalty_tier_name","loyalty_status")

In [10]:
#Checking dfTemplateDefinition File

if (dfTemplateDefinition
    .select("template_id","segment_module_map_id","module_priority_in_slot").distinct().count() == dfTemplateDefinition.count()):
    print ("TemplateDefinition File OK!!")
else:
    print ("Appending TemplateDefinition!!")
    dfTemplateDefinition = (dfTemplateDefinition
                            .groupBy("template_id","segment_module_map_id","module_priority_in_slot")
                            .agg({"slot_position":"min","placement_type":"first"})
                            .withColumnRenamed("first(placement_type)","placement_type")
                            .withColumnRenamed("min(slot_position)","slot_position"))

TemplateDefinition File OK!!


In [11]:
#dfTemplateDefinition.toPandas().to_csv("/home/affine/Downloads/Untitled Folder/unit_test_files/dfTemplateDefinition.csv")

## populating values in all rows for var source column

In [12]:
def coalesceCol(x,y):
    if (x != ""):
        return x
    else:
        return y
udfcoalesceCol = udf(coalesceCol, StringType())

module_var_def = dfModuleVariableDefinition

df2 = (module_var_def.select("module_id","source_connection","var_source")
       .filter("var_source != ''")
       .withColumnRenamed("var_source","var_source1"))   

df3 = module_var_def.join(df2,["module_id","source_connection"],'left')

dfModuleVariableDefinition = (df3
                        .withColumn("var_source_final", udfcoalesceCol(df3.var_source,df3.var_source1))
                        .distinct()
                        .drop("var_source").drop("var_source1")
                        .withColumnRenamed("var_source_final","var_source"))

In [13]:
#dfModuleVariableDefinition.toPandas().to_csv("/home/affine/Downloads/Untitled Folder/unit_test_files/dfModuleVariableDefinition.csv")

## creating meta campaign data by combining all the raw files

In [14]:
day_of_Week = 1   ###datetime.datetime.today().weekday() keep need to change it
dfMetaCampaignData1 = dfCampaignDefinition[(dfCampaignDefinition['dayofweek'] == day_of_Week+1)].orderBy("priority")


dfMetaCampaignData2 = (dfMetaCampaignData1.join(dfTemplateDefinition,'template_id','inner')
                                              .join(dfSegmentModule,'segment_module_map_id','inner')
                                              .join(dfModuleDefinition,['locale','module_type_id','tpid','eapid','placement_type','channel'],'inner')                                              
                                              .join(dfSegmentDefinition,["tpid","eapid","segment_type_id"],'inner'))

In [15]:
dfMetaCampaignData_31 = (dfMetaCampaignData2.select([c for c in dfMetaCampaignData2.columns if c not in 
                                                {"campaign_type_id","dayofweek","program_type",
                                                 "derived_module_id","context","language","lob_intent","status"}])
                                              )

In [16]:
#dfMetaCampaignData_31.toPandas().to_csv("/root/Expedia_DE/Data/projectalpha/data/module_type_test_files/dfMetaCampaignData_31.csv")

In [17]:
# correcting name of the column email_address

dfTravelers = (dfTraveler.withColumnRenamed("\ufeffemail_address","email_address").cache())


# Populating segment type id in travelers data:

In [18]:
dfMetaCampaign = (dfMetaCampaignData_31
                  .select("tpid","eapid","locale","segment_type_id","segment_criteria","campaign_id","priority")
                  .distinct()
                 )


In [19]:
## removing the segment criteria related to reward_redeem as this column is not present in travaler data

segment_criteria = (dfSegmentDefinition.filter("segment_type_name != 'reward_redeem'")
                    .select("segment_criteria").distinct()
                    .flatMap(lambda x:x)
                    .collect())


In [20]:
counter = 1

for i in segment_criteria:    
    criteria = (i.replace('[','').replace(']',''))    
    tmp = (dfTravelers.withColumn("segment_criteria",lit(i)).filter(criteria))
    
    if counter == 1:
        traveler = tmp
        counter+=1
    else:
        traveler = traveler.unionAll(tmp)


In [21]:
##changed the join from left to inner
dfTraveler_MetaCampaign = (traveler.join(dfMetaCampaign,["segment_criteria","tpid","eapid","locale"],'inner')
                            .distinct())

In [22]:
w = Window.partitionBy("email_address").orderBy("priority")

traveler_RU = (dfTraveler_MetaCampaign.withColumn("index", dense_rank().over(w))
                        .filter("index = 1").drop("index"))

In [23]:
traveler_RU_multi_row_treat = (traveler_RU
                             .groupBy('email_address')
                             .agg({'segment_type_id':'max','campaign_id':'first'})
                             .withColumnRenamed('max(segment_type_id)','segment_type_id')
                             .withColumnRenamed('first(campaign_id)','campaign_id')
                            )

In [24]:
final_df_for_alpha = (traveler_RU_multi_row_treat
                              .join(traveler_RU,['email_address','campaign_id','segment_type_id'],'inner')
                              .repartition(rep)
                              .cache()
                             )

In [25]:
#final_df_for_alpha.toPandas().to_csv("/root/Expedia_DE/Data/projectalpha/data/module_type_test_files/final_df_for_alpha.csv")

## Creating module version mapping

In [26]:
def dict_map(module_type_id,num_versions):
    import math
    
    seed = datetime.date.today().isocalendar()[1] 
    split = math.ceil((99/num_versions))
    
    rank_dict = {}
    for i in range(num_versions):
        rank = ((i) + seed%num_versions)%num_versions+1
        rank_dict[rank] = i+1

    range_dict = {}
    j = 1
    for key in rank_dict:
        range_dict[rank_dict[key]] = str(j)+'#'+str(j+split-1)
        j+= split
    
    return range_dict

dict_map_udf = udf(dict_map,MapType(IntegerType(),StringType()))

In [27]:
df_module = (dfMetaCampaignData_31
             .groupBy('module_type_id')
             .agg(countDistinct('version'))
             .withColumnRenamed('count(version)','num_versions')
            )

In [28]:
col_grp = ['campaign_id','slot_position','module_type_id']

dfMetaCampaignData_4 = (dfMetaCampaignData_31
                        .select(col_grp)
                        .join(df_module,'module_type_id','inner')
                        .distinct()
                        .withColumn('version_map',dict_map_udf('module_type_id','num_versions'))
                        .drop('num_versions').drop('travelers')
                        )

In [29]:
#dfMetaCampaignData_4.toPandas().to_csv("dfMetaCampaignData_4.csv")

## Joining module_var_definition with meta campaign data

In [30]:
#Joining with dfModuleDefinition to populate Module Priority and with dfSegmentsDefinition to fetch Segment Criteria
dfMetaCampaignData_VarDef = (dfMetaCampaignData_31
                         .join(dfModuleVariableDefinition,"module_id",'inner')
                         .join(dfMetaCampaignData_4,col_grp,'inner')
                         .drop("locale"))

In [31]:
#module_content_data_final.toPandas().to_csv("/root/Expedia_DE/Data/projectalpha/data/module_type_test_files/module_content_data_final.csv")

In [32]:
#selecting the required columns for module id allocation
col_list = ['campaign_id','slot_position','var_structure','var_source','var_position','module_id','module_type_id'
           ,'module_priority','segment_type_id','version','version_map','module_priority_in_slot']


dict_cpgn = sc.broadcast(dfMetaCampaignData_VarDef
                        .select(col_list).toPandas())

In [33]:
slot_position_map = (dfMetaCampaignData_VarDef
                     .groupBy('slot_position')
                     .agg({'var_position':'max'})
                     .withColumnRenamed('max(var_position)','var_position')
                     .rdd
                     .collectAsMap())

In [44]:
slots = dfTemplateDefinition.select('slot_position').distinct().count()

In [45]:
df_slot = final_df_for_alpha.withColumn('number_slots',lit(slots)) 

In [52]:
def fun(campaign_id,test_keys,segment_type_id,number_slots):
    
    import pandas as pd
    num_slots = number_slots
    map_dict1 = {}
    
    for i in range(1,(num_slots+1)):
        slot_position = i
        cpgn_sub = dict_cpgn.value[(dict_cpgn.value.campaign_id == campaign_id) &
                                  (dict_cpgn.value.slot_position == slot_position) ]

        cpgn_sub_final = cpgn_sub[(cpgn_sub['segment_type_id'] == segment_type_id)]

        if len(cpgn_sub_final) == 0:
            cpgn_sub_final = cpgn_sub[(cpgn_sub['segment_type_id'] == 1)]


        test = pd.DataFrame()

        module_types = list(cpgn_sub_final.module_type_id.unique())

        for module in module_types:

            version_sel = cpgn_sub_final[(cpgn_sub_final.module_type_id == module)].iloc[0].version_map

            for key in version_sel:

                ll = int(version_sel[key].split('#')[0])
                ul = int(version_sel[key].split('#')[1])

                if (test_keys >=ll) & (test_keys <=ul):
                    test = test.append(cpgn_sub_final[(cpgn_sub_final.version==key) & (cpgn_sub_final.module_type_id == module)])


        col_req = ['module_type_id','module_priority']

        test.index = (test[col_req]
                      .astype(str)
                      .apply(lambda x : '#'.join(x), axis=1))

        test['module_id'] = test['module_id'].astype('str')

        final_dict = test[['module_id']].to_dict()['module_id'] 
        map_dict1[str(i)] = final_dict
    
    return map_dict1

fun_udf = udf(fun,MapType(StringType(),MapType(StringType(),StringType())))


In [54]:
df_slot_fun = df_slot.withColumn('map_dict1',fun_udf('campaign_id','test_keys','segment_type_id','number_slots'))

In [78]:
def module_info(map_dict1):
    map_dict = {}
    for slot in map_dict1:
        for key in map_dict1[slot]:
            module_id = map_dict1[slot][key]
            test = dict_cpgn.value[dict_cpgn.value.module_id == int(module_id)][['var_source','var_position','var_structure']]
            col_req = ['var_source','var_position']
            test.index = (test[col_req]
                          .astype(str)
                          .apply(lambda x : '#'.join(x), axis=1))
            test_dict = test[['var_structure']].to_dict()['var_structure']
            map_dict[str(module_id)] = test_dict

    return map_dict
    
module_info_udf = udf(module_info,MapType(StringType(),MapType(StringType(),StringType())))

In [79]:
df_slot_fun_module = df_slot_fun.withColumn('map_dict',module_info_udf('map_dict1'))

## populating the value in columns created above

In [81]:
def content_map(row):
    
    import re
    dict_req = {'email_address':row['email_address']}
    number_slots = row['number_slots']
    for slot_position in range(1,number_slots+1):
    
        map_dict1 = row['map_dict1'][str(slot_position)]
        slot_position = str(slot_position)
        df = pd.DataFrame()

        for key1 in  map_dict1:

            map_dict = row['map_dict'][map_dict1[key1]]

            var_positions = slot_position_map[int(slot_position)]

            x_dict = {('S'+slot_position+'_P'+str(v)):'' for v in range(1,var_positions+1)}

            x_dict['S'+slot_position+'_module_priority'] = int(key1.split('#')[1])

            for key in map_dict:

                join_key =  key.split('#')[0]

                slot_var_pos = 'S'+slot_position+'_P'+str(key.split('#')[1]) #Assigning the Slot_Pos Name

                if( len(join_key.split('|')) > 1):
                    file_name =  join_key.split('|')[1].split(';')[0].split('.')[0]
                    left_keys = [col.split('.')[1] for col in join_key.split('|')[0].split(';')]
                    right_keys = [col.split('.')[1] for col in join_key.split('|')[1].split(';')]

                else :
                    file_name =  'NA'
                    left_keys = 'NA'
                    right_keys = 'NA'


                if (len(map_dict[key]) == 0):
                    var_value = ""

                elif(len(map_dict[key].split('%%')) == 1):
                     var_value = map_dict[key]

                else : 
                    z = [m.start() for m in re.finditer('%%', map_dict[key])]
                    var_value = map_dict[key]

                    j = 0
                    while (j <= len(z)-2):
                        temp = map_dict[key][z[j]+2:z[j+1]]
                        df_name = temp.split('.')[0]


                        if (df_name == 'traveler'):

                            trav_attri = row[temp.split('.')[1]]

                            if trav_attri == '':
                                trav_attri = "value_not_found"

                            var_value = var_value.replace(temp,trav_attri)

                        else :

                            req_content = frames_dict[df_name].value

                            for i in range(len(right_keys)):
                                req_content = req_content[req_content[right_keys[i]]==row[left_keys[i]]]

                            if len(req_content)==0:
                                x_dict['S'+slot_position+'_module_priority'] = 9999

                            content_ls = list(req_content[temp.split('.')[1]])

                            if len(content_ls) > 0:
                                value_re = content_ls[0]
                                if value_re == '':
                                    value_re = "value_not_found"
                            else : 
                                value_re = "value_not_found"
                                x_dict['S'+slot_position+'_module_priority'] = 9999

                            var_value = var_value.replace(temp,str(value_re))

                        j+=2

                x = var_value.replace('%%','')

                ttl_options = len(x.split("|"))
                seed = datetime.date.today().isocalendar()[1] 

                if(ttl_options > 2):
                    att = int((row['test_keys']+ seed%(ttl_options-1))%(ttl_options-1)) ####
                elif (ttl_options == 2):
                    att = int((row['test_keys']+ seed%(ttl_options))%(ttl_options)) ####
                else:
                    att = 0

                if ((x.lower().find('value_not_found') > 0)):
                    x_dict[slot_var_pos] = x.split('|')[0].replace('value_not_found','')

                else:
                    if (ttl_options<=1):
                        x_dict[slot_var_pos] = x.split('|')[0].replace('value_not_found','')
                    elif (ttl_options==2):
                        list_value = x.split('|')
                        x_dict[slot_var_pos]=(list_value[att]).replace('value_not_found','')
                    else :
                        list_value = x.split('|')[1:]
                        x_dict[slot_var_pos]=(list_value[att]).replace('value_not_found','')


                x_dict['S'+slot_position+'_module_id'] = map_dict1[key1]
                x_dict['S'+slot_position+'_module_type_id'] = str(key1.split('#')[0])


            temp = pd.DataFrame(x_dict,index=[x_dict['S'+slot_position+'_module_priority']])

            df = df.append(temp)

        final_df = df.loc[[df['S'+slot_position+'_module_priority'].idxmin()]]      
        col_name = list(final_df['S'+slot_position+'_module_priority'])[0]

        mod_id = final_df['S'+slot_position+'_module_id'].iloc[0]

        for prior in map_dict1:
            if map_dict1[prior] == mod_id:
                module_prior = prior.split('#')[1]

        final_df['S'+slot_position+'_module_priority'] =  module_prior
        final_dict = final_df.transpose().to_dict()[col_name]

        dict_req.update(final_dict)
    
    return Row(**dict_req)


In [83]:
final_data = df_slot_fun_module.rdd.map(lambda x : content_map(x)).toDF().repartition(rep,'email_address').cache()

In [84]:
final_result = final_df_for_alpha.join(final_data,'email_address','inner').cache()

In [85]:
final_result.count()

300

In [42]:
#final_result.toPandas().to_csv('final_result_v5.8.csv')